In [ ]:
import watertap as wt

In [ ]:
from case_study_trains import *
import case_study_trains

In [ ]:
m = wt.watertap_setup(dynamic = False)

In [ ]:
wt.case_study_trains.train = {"case_study": "Carlsbad",
                             "reference": "NAWI",
                             "scenario": "Baseline"}

In [ ]:
# TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#But, if the source water type is different to what is in the train (pfd dictionary), 
#then we should updat the node name. If more than two sources - what to do? Needs to be
#based on pfd node!?

wt.case_study_trains.source_water = {"case_study": "Carlsbad", 
                             "reference": "NAWI",
                             "scenario": "Baseline",
                             "water_type": "Seawater"}

In [ ]:
m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.

In [7]:
wt.display.show_train2(model_name=m)

In [8]:
wt.run_water_tap(m = m, solver_results = True, print_model_results = True)

degrees_of_freedom: 0
Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2547
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      614

Total number of variables............................:      818
                     variables with only lower bounds:       34
                variables with lower and upper bounds:        2
                     variables with only uppe

    None :   conc_mass : {(0.0, 'bromide'): -3.5130553656328933e-44, (0.0, 'calcium'): -3.5067324722772493e-44, (0.0, 'chloride'): -3.149096476136806e-44, (0.0, 'magnesium'): -3.5338466295214733e-44, (0.0, 'potassium'): -3.5192686940921307e-44, (0.0, 'sodium'): -4.04703670709924e-44, (0.0, 'strontium'): -3.5170494724146644e-44, (0.0, 'sulfate'): -3.569996216540406e-44, (0.0, 'tds'): -5.092105311317861e-44}
         :    flow_vol : {0.0: 1.6648008681346253e-19}
         :    pressure : {0.0: 199999.99854545455}
         : temperature : {0.0: 299.9997181818182}
total_cap_investment: 52.33757013302066
----------------------------------------------------------------------
----------------------------------------------------------------------
fs.municipal_drinking
inlet : Size=1
    Key  : Name        : Value
    None :   conc_mass : {(0.0, 'bromide'): 1.2523333340473548, (0.0, 'calcium'): 7.736888893335675, (0.0, 'chloride'): 361.94888909721925, (0.0, 'magnesium'): 24.140000013940426, (0.0

In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
cost_range_list

In [ ]:
#### DO NOT USE THE BELOW ####

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)